# Cleaning string data
Data cleaning is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate, or incomplete data within a dataset.
For strings this process starts by applying a list of common functions (lower, strip).  Then handling missing and duplicate entries. And finally applying custom functions as needed.

In [191]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Set max rows and columns displayed in jupyter
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)

## Load a dataset

In [192]:
df=pd.read_csv('../projects/proj1/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv')
df.head()
print(df.shape)

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


(27609, 18)


In [193]:
x=df['Job title'].unique()
x.sort()
x[:50]

array([' Analyst', ' Business Systems Analyst', ' CAP team',
       ' Cloud Architect ', ' Commercial Manager ',
       ' Content Development Specialist', ' Controller ',
       ' Customer Solutions Manager', ' Customer Support Supervisor',
       ' Field Associate', ' Industrial Hygienist',
       ' Network Coordinator', ' Nurse Practitioner', ' Program Manager ',
       ' Senior DevOps Engineer', ' Senior Manager',
       ' Technical Systems Administrator', ' Vendor Manager', ' clerical',
       '"mum" ;)', '(Electronics) Hardware engineer',
       '(Environmental) Planner', '(Hotel) Sales Manager',
       '(State Agency) Legislative Director', '-', '1st Grade Teacher',
       '1st Line ICT Support Analyst', '1st line manager',
       '1x1 coordinator', '24 Frame Video Playback', '2D Artist',
       '2nd Grade Teacher', '3D Animator', '3D Artist',
       '3D Character Artist', '3D Project Coordinator', '3d artist',
       '3rd Deputy-County Treasurer’s Office',
       '3rd line techn

In [194]:
df.dtypes

Timestamp                                    object
How old are you?                             object
Industry                                     object
Job title                                    object
Additional context on job title              object
Annual salary                                object
Other monetary comp                         float64
Currency                                     object
Currency - other                             object
Additional context on income                 object
Country                                      object
State                                        object
City                                         object
Overall years of professional experience     object
Years of experience in field                 object
Highest level of education completed         object
Gender                                       object
Race                                         object
dtype: object

## Lets handle missing Country data (it will mess up below algorithms).  
Lots of ways to do this, I'm going to do it the simple way, replace NaN with UNKNOWN

This is not a good idea in general, especially if you have a way to figure out the country from the other data present

In [195]:
#how many missing Countries
df.Country.isnull().sum()

0

In [196]:
df.Country = df.Country.fillna('UNKNOWN')

## There is a country column, lets use it to get all the USA entries.  Take a look at the number of unique entries

In [197]:
#how many different countries are there
df.Country.nunique()

364

## How many occurrences for each unique entry?

In [198]:
#lets see what we have
vc=df.Country.value_counts()
print(f'There are {len(vc)} unique entries')
vc[:50]
# vc[50:100]

There are 364 unique entries


United States                8844
USA                          7847
US                           2572
Canada                       1549
United States                 652
U.S.                          571
UK                            566
United Kingdom                540
USA                           468
Usa                           441
United States of America      421
Australia                     312
United states                 203
usa                           180
Germany                       168
England                       134
united states                 113
Us                            103
Ireland                       102
New Zealand                    94
Uk                             84
Canada                         75
Australia                      67
United Kingdom                 65
France                         63
U.S.A.                         46
United States of America       43
Spain                          40
Netherlands                    40
Scotland      

## It looks like there was no filtering on what a user could enter in this field, ANyway lets get to it

## Apply lower and strip to get the easy gains

In [199]:
df.Country = df.Country.map(str.lower).map(str.strip)

#how many different countries are there now
df.Country.nunique()

250

## Looks like a lot of punctuation, lets get rid of it
Use regular expressions

In [200]:
#the regular expressions package
import re
punc = "[!\"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~\`]"  #this is the punctuation to get rid of

#or a function
def fun1(x):
    #re.sub will remove any punction char found in punc 
    return re.sub(punc, '', x)

df.Country = df.Country.map(fun1)

#can do the same thing with a lambda
# df.Country = df.Country.map(lambda x: re.sub(punc, '', x))

#how many different countries are there now
df.Country.nunique()

236

## Looks like a lot of variations of 'united state'

Another easy gain, lets replace all strings with 'united state' in them with 'usa' 

In [201]:
def fun(x):
    """
    replaces any string that contains 'united state' with 'usa'
    """
    if 'united state' in x:
        return 'usa'
    return x


df.Country = df.Country.map(fun);

#how many different countries are there now
df.Country.nunique()

219

## But you might want to do something similar with other strings, do you write another function?  Or do something a little more general?
Be general, always, use a python closure.

## Closures
The problem we face is that map takes a function that takes 1 parameter, and we want it to take 3; The string value passed by map (call it x), the string to search for in x (call it str_to_find), and the string to replace x with if we find str_to_find in x (call it str_replacement).

We can't get around the fact that map only passes 1 parameter to the function.  But we can creae a function that  already knows what str_to_find and str_replacement are.  Its called a closure

In [202]:
def fun1(str_to_find, str_replacement):
    """
    creates findandreplace Closure, which is a stateful function
    that remembers str_to_find and str_replacement values
    returns: findandreplace
    """
    def findandreplace(x):
        if str_to_find in x:
            return str_replacement
        return x
    # in python functions are first class objects
    # we are returning findandreplace, it in turn knows the value of 
    # str_to_find and str_replacement
    return findandreplace


In [203]:
#using the closure
fn= fun1('usa', 'usa')
df.Country = df.Country.map(fn)

fn= fun1('us', 'usa')
df.Country = df.Country.map(fn)

fn= fun1('u s', 'usa')
df.Country = df.Country.map(fn)

fn= fun1('unites states', 'usa')  #its a bit suspicious that 17 people made this mistake
df.Country = df.Country.map(fn)

fn= fun1('united sates', 'usa')
df.Country = df.Country.map(fn)

fn= fun1('unitedstates', 'usa')
df.Country = df.Country.map(fn)

fn= fun1('united stares', 'usa')
df.Country = df.Country.map(fn)
#and so on

In [204]:
#you can simplify the above with a list of str_to_find
#and just iterate over it
vals=['usa', 'us', 'u s', 'unites states', 'united sates', 'unitedstates', 'united stares']
for val in vals:
    fn= fun1(val, 'usa')
    df.Country = df.Country.map(fn)


In [205]:
#how many different countries are there now
df.Country.nunique()

182

## Once you get down to the bottom of the unique values you will probably get a lot of one offs
For instance lets see what the the values are

In [206]:
vals=df.Country.unique()
vals.sort()
vals

array(['217584year is deducted for benefits', 'afghanistan', 'africa',
       'america', 'aotearoa new zealand', 'argentina',
       'argentina but my org is in thailand', 'bangladesh', 'belgium',
       'bermuda', 'brasil', 'brazil', 'britain', 'bulgaria', 'california',
       'cambodia', 'can', 'canad', 'canada', 'canada ottawa ontario',
       'canadw', 'canadá', 'canda', 'catalonia', 'cayman islands',
       'chile', 'china', 'colombia',
       'company in germany i work from pakistan', 'congo', 'contracts',
       'costa rica', 'cote divoire', 'croatia', 'csnada', 'cuba',
       'currently finance', 'czech republic', 'czechia', 'danmark',
       'denmark', 'ecuador', 'england', 'england gb', 'england uk',
       'england united kingdom', 'englanduk', 'englang', 'eritrea',
       'estonia', 'europe', 'finland', 'france',
       'from new zealand but on projects across apac', 'germany', 'ghana',
       'global', 'great britain', 'greece', 'hartford', 'hong kong',
       'hong konh',

## Notice there are a lot of united states ish entries at the bottom, lets see if fuzzy wuzzy helps

In [207]:
# this package lives in the conda forge
# !conda install -c conda-forge fuzzywuzzy -y

In [208]:
# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process

In [209]:
#first is the match, second is the score, third is the index in the dataframe
matches = fuzzywuzzy.process.extract("unit", df.Country, limit=50, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('united y', 67, 22822),
 ('kuwait', 60, 1915),
 ('unite states', 50, 10447),
 ('unite states', 50, 13362),
 ('united sttes', 50, 14113),
 ('united statws', 47, 2003),
 ('united kindom', 47, 7156),
 ('uniter statez', 47, 16890),
 ('unitef stated', 47, 20206),
 ('united statss', 47, 25473),
 ('united  states', 47, 26059),
 ('lithuania', 46, 11426),
 ('lithuania', 46, 20695),
 ('united kingdom', 44, 1),
 ('united kingdom', 44, 15),
 ('united kingdom', 44, 59),
 ('united kingdom', 44, 93),
 ('united kingdom', 44, 114),
 ('united kingdom', 44, 126),
 ('united kingdom', 44, 147),
 ('united kingdom', 44, 177),
 ('united kingdom', 44, 207),
 ('united kingdom', 44, 295),
 ('united kingdom', 44, 301),
 ('united kingdom', 44, 339),
 ('india', 44, 434),
 ('united kingdom', 44, 493),
 ('united kingdom', 44, 518),
 ('united kingdom', 44, 641),
 ('united kingdom', 44, 648),
 ('united kingdom', 44, 664),
 ('united kingdom', 44, 682),
 ('united kingdom', 44, 699),
 ('united kingdom', 44, 785),
 ('uni

In [210]:
#get the first match to prove that the index is the index in the dataframe
df.iloc[ matches[0][2]].Country

'united y'

In [211]:
#they look pretty good, except for the united kingdom, kuwait (kuwait???), india
#lets get all the unique matches in matches
#first all matches
l=[]
for mtch in matches:
    l.append(mtch[0])


# get unique values
l = set(l)
print(l)

{'united statws', 'unite states', 'united  states', 'india', 'lithuania', 'united kingdom', 'unitef stated', 'united y', 'uniter statez', 'kuwait', 'united statss', 'united kindom', 'united sttes'}


In [212]:
#create a function that will sub in 'usa' for the remaining matches
def subinmatches(df,mtches, dont_sub_these, str_replacement):
    for mtch in mtches:
        if(mtch[0] in dont_sub_these):
            continue
        df.iloc[mtch[2],df.columns.get_loc('Country')]=str_replacement

#gleaned from previous cells print
dont_sub_these=['lithuania','kuwait','united kingdom', 'india', 'united kindom']
subinmatches(df,matches,dont_sub_these,'usa')    

#how many different countries are there now
df.Country.nunique()

174

# Lets see if we can do this for other countries.  It will be easier if we have a package that has a list of countries.

<a href="https://pypi.org/project/pycountry/">pycountry</a> is perfect.  It has a list of countries and a fuzzy lookup API. 


In [213]:
# !conda install -c conda-forge pycountry -y

In [214]:
import pycountry

In [223]:
vc=df.Country.value_counts()

# create a df to hold above name and count.  
# Add a column to track whether the string 
# in df.Country has been pycountry matched to an acual country.
cdf=vc.reset_index()
cdf.rename(columns = {'Country':'numb_entries'},inplace=True)
cdf.rename(columns = {'index':'Country'},inplace=True)
cdf['found_in_db']=False
cdf

,Country,numb_entries,found_in_db
0,usa,23217,False
1,canada,1656,False
2,uk,695,False
3,united kingdom,626,False
4,germany,190,False
...,...,...,...
169,canada ottawa ontario,1,False
170,cambodia,1,False
171,canad,1,False
172,global,1,False


### The following pycountry call will return a tupple if the country found, otherwise it will throw a LookupError

In [224]:
pycountry.countries.lookup('de')
pycountry.countries.lookup('germany')
pycountry.countries.lookup('usa').alpha_3
pycountry.countries.lookup('United States of America')
# pycountry.countries.lookup('united sates') #fails

Country(alpha_2='DE', alpha_3='DEU', name='Germany', numeric='276', official_name='Federal Republic of Germany')

Country(alpha_2='DE', alpha_3='DEU', name='Germany', numeric='276', official_name='Federal Republic of Germany')

'USA'

Country(alpha_2='US', alpha_3='USA', name='United States', numeric='840', official_name='United States of America')

In [94]:
cdf.sort_values(by='found_in_db')[-100:]

,index,Country,found_in_db
42,wales,4,False
140,united stattes,1,False
39,great britain,5,False
38,south korea,5,False
36,england uk,5,False
104,méxico,1,False
144,uniyes states,1,False
145,csnada,1,False
146,uk northern ireland,1,False
115,ecuador,1,True


In [124]:
#lets try a fuzzy search to seee if we get a few more
pycountry.countries.search_fuzzy('wales')

[Country(alpha_2='GB', alpha_3='GBR', name='United Kingdom', numeric='826', official_name='United Kingdom of Great Britain and Northern Ireland'),
 Country(alpha_2='AU', alpha_3='AUS', name='Australia', numeric='036')]

In [226]:
def fix(df, i, cont, search_type):
    print(f' {search_type} {cont.alpha_3} for {df.loc[i, "Country"]}')
    df.loc[i, 'found_in_db'] = True  #found column
    df.loc[i, 'Country']=cont.alpha_3
def find_countries(df):
    #record whether pycountry finds result or not
    for i in range(len(df)):
        # print(cdf.iloc[i, 0])
        try:
            #try the most accurate lookup first
            res = pycountry.countries.lookup(df.loc[i, 'Country']) 
            fix(df, i, res, "Accurate swap")
        except LookupError as le:
            #accurate failed try fuzzy
            try:
                res = pycountry.countries.search_fuzzy(df.loc[i, 'Country'])
                fix(df, i, res[0], "Fuzzy swap")
            except LookupError as le:
                pass
    return df

cdf = find_countries(cdf)
# df = find_countries(df)

 Accurate swap USA for usa
 Accurate swap CAN for canada
 Fuzzy swap UKR for uk
 Accurate swap GBR for united kingdom
 Accurate swap DEU for germany
 Fuzzy swap GBR for england
 Accurate swap IRL for ireland
 Accurate swap NZL for new zealand
 Accurate swap FRA for france
 Accurate swap NLD for netherlands
 Fuzzy swap GBR for scotland
 Accurate swap ESP for spain
 Accurate swap SWE for sweden
 Accurate swap CHE for switzerland
 Accurate swap BEL for belgium
 Fuzzy swap NLD for the netherlands
 Accurate swap JPN for japan
 Accurate swap DNK for denmark
 Fuzzy swap ASM for america
 Accurate swap ZAF for south africa
 Accurate swap FIN for finland
 Accurate swap ISR for israel
 Accurate swap ITA for italy
 Accurate swap SGP for singapore
 Accurate swap IND for india
 Accurate swap NOR for norway
 Accurate swap MYS for malaysia
 Accurate swap BRA for brazil
 Accurate swap PHL for philippines
 Accurate swap POL for poland
 Accurate swap MEX for mexico
 Accurate swap CHN for china
 Accurate 

In [231]:
def print_stats(cdf):
    # how many found?
    numb_found = cdf.found_in_db.sum()
    print(f' Number countries identified {numb_found}')
    print(f' Number countries unidentified {len(cdf)-numb_found}')
    print(f' Number unique countries {df.Country.nunique()}')

print_stats(cdf)

 Number countries identified 111
 Number countries unidentified 63
 Number unique countries 174


In [242]:
# cdf.groupby('found_in_db').sum().loc[False]
cdf.groupby('found_in_db').sum()

,numb_entries
found_in_db,
False,82
True,27527


In [243]:
# only 82 remaining unknown countries out of 27000
# lets see them
cdf.sort_values(by='found_in_db')[:85]

,Country,numb_entries,found_in_db
48,england united kingdom,3,False
66,canda,2,False
113,luxemburg,1,False
114,san francisco,1,False
62,wales uk,2,False
117,i earn commission on sales if i meet quota im ...,1,False
120,uxz,1,False
121,hartford,1,False
67,remote,2,False
122,new zealand aotearoa,1,False
